In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary
import random
import numpy as np

In [ ]:
random.seed(1)
np.random.seed(1)

## Orignal architecture

In [2]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input 28x28x1 Output 28x28x32 RF 3
#         self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #input 28x28x32 Output 28x28x64 RF 5
#         self.pool1 = nn.MaxPool2d(2, 2) #input 28x28x64 Output 14x14x64 RF 6
#         self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input 14x14x64 Output 14x14x128 RF 10
#         self.conv4 = nn.Conv2d(128, 256, 3, padding=1) #input 14x14x128 Output 14x14x256 RF 14
#         self.pool2 = nn.MaxPool2d(2, 2) #input 14x14x256 Output 7x7x256 RF 16
#         self.conv5 = nn.Conv2d(256, 512, 3) #input 7x7x256 Output 5x5x512 RF 24
#         self.conv6 = nn.Conv2d(512, 1024, 3) #input 5x5x512 Output 3x3x1024 RF 28
#         self.conv7 = nn.Conv2d(1024, 10, 3) #input 3x3x1024 Output 1x1x10 RF 36

#     def forward(self, x):
#         x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
#         x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
#         x = F.relu(self.conv6(F.relu(self.conv5(x))))
#         x = self.conv7(x)
#         x = x.view(-1, 10)
#         return F.log_softmax(x)

## 20K parameter Architecture (GAP and kernels range 8 to 32)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1, bias=False) #input 28x28x1 Output 28x28x32 RF 3
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1, bias=False) #input 28x28x32 Output 28x28x64 RF 5
        self.pool1 = nn.MaxPool2d(2, 2) #input 28x28x64 Output 14x14x64 RF 6
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1, bias=False) #input 14x14x64 Output 14x14x128 RF 10
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1, bias=False) #input 14x14x128 Output 14x14x256 RF 14
        self.conv4_1 = nn.Conv2d(32, 8, 1) #input 14x14x128 Output 14x14x256 RF 14
        self.pool2 = nn.MaxPool2d(2, 2) #input 14x14x256 Output 7x7x256 RF 16
        self.conv5 = nn.Conv2d(8, 16, 3, bias=False) #input 7x7x256 Output 5x5x512 RF 24
        self.conv5_1 = nn.Conv2d(16, 16, 3, bias=False) #input 7x7x256 Output 5x5x512 RF 24
        self.gap6 = nn.AdaptiveAvgPool2d((1, 1)) #input 5x5x512 Output 3x3x1024 RF 28
        self.conv7 = nn.Conv2d(16, 10, 1) #input 3x3x1024 Output 1x1x10 RF 36

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4_1(F.relu(self.conv4(F.relu(self.conv3(x)))))))
        x = F.relu(self.gap6(F.relu(self.conv5_1(F.relu(self.conv5(x))))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

## Batch Normalization

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1, bias=False) #input 28x28x1 Output 28x28x32 RF 3
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1, bias=False) #input 28x28x32 Output 28x28x64 RF 5
        self.bn2 = nn.BatchNorm2d(16)
        
        self.pool1 = nn.MaxPool2d(2, 2) #input 28x28x64 Output 14x14x64 RF 6
        
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1, bias=False) #input 14x14x64 Output 14x14x128 RF 10
        self.bn3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1, bias=False) #input 14x14x128 Output 14x14x256 RF 14
        self.bn4 = nn.BatchNorm2d(32)
        self.conv4_1 = nn.Conv2d(32, 8, 1) #input 14x14x128 Output 14x14x256 RF 14
        self.bn4_1 = nn.BatchNorm2d(8)

        self.pool2 = nn.MaxPool2d(2, 2) #input 14x14x256 Output 7x7x256 RF 16
        
        self.conv5 = nn.Conv2d(8, 16, 3, bias=False) #input 7x7x256 Output 5x5x512 RF 24
        self.bn5 = nn.BatchNorm2d(16)
        self.conv5_1 = nn.Conv2d(16, 16, 3, bias=False) #input 7x7x256 Output 5x5x512 RF 24
        self.bn5_1 = nn.BatchNorm2d(16)
        
        self.gap6 = nn.AdaptiveAvgPool2d((1, 1)) #input 5x5x512 Output 3x3x1024 RF 28
        self.conv7 = nn.Conv2d(16, 10, 1) #input 3x3x1024 Output 1x1x10 RF 36

    def forward(self, x):
        x = self.pool1(self.bn2(F.relu(self.conv2(self.bn1(F.relu(self.conv1(x)))))))
        x = self.pool2(self.bn4_1(F.relu(self.conv4_1(self.bn4(F.relu(self.conv4(self.bn3(F.relu(self.conv3(x))))))))))
        x = F.relu(self.gap6(self.bn5_1(F.relu(self.conv5_1(self.bn5(F.relu(self.conv5(x))))))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [5]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 16, 28, 28]           1,152
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
            Conv2d-6           [-1, 32, 14, 14]           4,608
       BatchNorm2d-7           [-1, 32, 14, 14]              64
            Conv2d-8           [-1, 32, 14, 14]           9,216
       BatchNorm2d-9           [-1, 32, 14, 14]              64
           Conv2d-10            [-1, 8, 14, 14]             264
      BatchNorm2d-11            [-1, 8, 14, 14]              16
        MaxPool2d-12              [-1, 8, 7, 7]               0
           Conv2d-13             [-1, 16, 5, 5]           1,152
      BatchNorm2d-14             [-1, 1

/home/gauravp/anaconda3/envs/kerasretina/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [6]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [7]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'epoch = {epoch} loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
num_epochs = 20
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0659, Accuracy: 9819/10000 (98.1900%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0401, Accuracy: 9891/10000 (98.9100%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0220, Accuracy: 9938/10000 (99.3800%)



epoch = 19 loss=0.0010424405336380005 batch_id=468: 100%|██████████| 469/469 [00:06<00:00, 75.20it/s] 
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0238, Accuracy: 9934/10000 (99.3400%)



epoch = 20 loss=0.02048623003065586 batch_id=468: 100%|██████████| 469/469 [00:06<00:00, 74.34it/s]   



Test set: Average loss: 0.0159, Accuracy: 9954/10000 (99.5400%)

